<a href="https://colab.research.google.com/github/kalai444/Dynamic-Neural-Networks-with-Early-Exit-Mechanisms-/blob/main/Dynamic_Neural_networks_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models,losses, optimizers
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from google.colab import files

In [ ]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar10.load_data()
x_train,x_test = x_train/225,x_test/255
y_train,y_test = y_train.flatten(),y_test.flatten()


In [ ]:
def build_dynamic_model(num_classes=10):
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(32, 3, activation='relu',padding='same')(inputs)
    x = layers.Conv2D(32, 3, activation='relu',padding='same')(inputs)
    x = layers.MaxPooling2D()(x)
    x1 = layers.GlobalAveragePooling2D()(x)
    exit1 = layers.Dense(64, activation='relu')(x1)
    exit1 = layers.Dense(num_classes, activation='softmax', name='exit1')(exit1)
    x = layers.Conv2D(64, 3, activation='relu',padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x2 = layers.GlobalAveragePooling2D()(x)
    exit2 = layers.Dense(64, activation='relu')(x2)
    exit2 = layers.Dense(num_classes, activation='softmax', name='exit2')(exit2)
    x = layers.Conv2D(64, 3, activation='relu',padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x3 = layers.GlobalAveragePooling2D()(x)
    exit3 = layers.Dense(64, activation='relu')(x3)
    exit3 = layers.Dense(num_classes, activation='softmax', name='exit3')(exit3)
    x = layers.Conv2D(128, 3, activation='relu',padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation='relu')(x)
    final_output = layers.Dense(num_classes, activation='softmax', name='final')(x)
    model = tf.keras.Model(inputs=inputs, outputs=[exit1, exit2, exit3, final_output])
    return model

model = build_dynamic_model()

model.compile(
    optimizer='adam',
    loss=[losses.SparseCategoricalCrossentropy(),
          losses.SparseCategoricalCrossentropy(),
          losses.SparseCategoricalCrossentropy(),
          losses.SparseCategoricalCrossentropy()],
    loss_weights=[0.1, 0.2,0.3, 0.4],
    metrics=['accuracy','accuracy','accuracy','accuracy']
)

model.fit(x_train, [y_train, y_train, y_train,y_train],
          epochs=10, batch_size=64,
          validation_split=0.1)
model.save('my_model_loss_4blocks_1.keras')

In [ ]:
model = tf.keras.models.load_model('my_model_loss_4blocks_1.keras')
# model.summary()

In [ ]:
def early_exit_predict(model, x, threshold):
    predictions = model.predict(x)
    exit_layers = []
    for i in range(len(x)):
        exited = False
        for j, pred in enumerate(predictions):
            conf = np.max(pred[i])
            if conf >= threshold:
              # exit_2_3.append(x[i])
                # exit_2_3.append(x[i],f'{j+1}')
                # I = Image.fromarray((x[i]*255).astype(np.uint8))
                # I = I.save(f'{j + 1}' + f'{i}'+'.png')
                # files.download(f'{j+1}'+f'{i}'+'.png')
              exit_layers.append(j + 1)
              exited = True
              break
        if exited == False:
          exit_layers.append(5)
    return exit_layers
exit_layers = early_exit_predict(model, x_test[:1000], threshold=0.70)



In [ ]:
exit_layers = early_exit_predict(model, x_test[:1000], threshold=0.70)
hard_indices = [i for i, exit in enumerate(exit_layers) if exit == 4]
hard_images = x_test[hard_indices]
hard_labels = y_test[hard_indices]
easy_indices = [i for i, exit in enumerate(exit_layers) if exit == 1 or exit==2 or exit== 3]
easy_images = x_test[easy_indices]
easy_labels = y_test[easy_indices]
# print(len(hard_indices))
# print(len(easy_indices))

In [ ]:
import matplotlib.pyplot as plt

def show_images(images, labels, title, n=12):
    plt.figure(figsize=(15, 3))
    for i in range(n):
        plt.subplot(1, n, i+1)
        plt.imshow(images[i])
        plt.title(f"Label: {labels[i]}")
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

show_images(hard_images, hard_labels, "Hard Images")
show_images(easy_images, easy_labels, "Easy Images")


In [ ]:
features_model = tf.keras.Model(inputs=model.input, outputs=model.get_layer('conv2d_7').output)
easy_features = features_model.predict(easy_images)
hard_features = features_model.predict(hard_images)


In [ ]:
one_easy_image = easy_images[0]  # shape: (32, 32, 3)
one_hard_image = hard_images[0]  # shape: (32, 32, 3)
one_easy_image_batch = np.expand_dims(one_easy_image, axis=0)
one_hard_image_batch = np.expand_dims(one_hard_image, axis=0)
feature_oneimage_E = features_model.predict(one_easy_image_batch)
feature_oneimage_H = features_model.predict(one_hard_image_batch)
mean_activation_E = np.mean(feature_oneimage_E)
mean_activation_H = np.mean(feature_oneimage_H)
print("Mean activation for single easy image:", mean_activation_E)
print("Mean activation for single easy image:", mean_activation_H)


In [ ]:
easy_mean_features = np.mean(easy_features, axis=(1,2,3))
hard_mean_features = np.mean(hard_features, axis=(1,2,3))
print(easy_mean_features)
print(hard_mean_features)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.hist(easy_mean_features, bins=30, alpha=0.6, label='Easy', color='green')
plt.hist(hard_mean_features, bins=30, alpha=0.6, label='Hard', color='red')
plt.title("Histogram of Mean Activations")
plt.xlabel("Mean Activation Value")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.hist(exit_layers,bins=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5],  rwidth=0.8)
plt.xticks([1,2,3,4,5], ['Exit 1', 'Exit 2','Exit 3','Final','Not met the threshold'])
plt.title("Exit Layer Distribution (Confidence ≥ 0.7)")
plt.xlabel("Exit Point")
plt.ylabel("Number of Images")
plt.show()